#### 1.1 Reorganize all csv files.

In [10]:
import os
import shutil
from glob import glob  
from tqdm import tqdm
from pathlib import Path

data_root = r".\Data"
directories = glob(os.path.join(data_root, r'城市*'))
print(directories)
for directory in tqdm(directories):
    if os.path.exists(os.path.join(directory, Path(directory).name)):
        inner_directory = os.path.join(directory, Path(directory).name)
        xmls = glob(os.path.join(inner_directory, '*.csv'))
        for xml in xmls:
            shutil.move(xml, 
                        os.path.join(directory, Path(xml).name.replace("china_cities_", "")))
        os.remove(inner_directory)
    else:
        xmls = glob(os.path.join(directory, '*.csv'))
        for xml in xmls:
            os.rename(xml, xml.replace("china_cities_", ""))
    os.rename(directory, os.path.join(Path(directory).parent, Path(directory).name[3:7]))

['.\\Data\\城市_20250101-20250329']


100%|██████████| 1/1 [00:00<00:00, 142.03it/s]


#### 1.2 Get a directory tree.

In [ ]:
from rich.tree import Tree
from rich import print
import os

def build_tree(path, tree):
    for item in sorted(os.listdir(path)):
        full_path = os.path.join(path, item)
        if os.path.isdir(full_path):
            branch = tree.add(f"[bold blue]{item}/")
            build_tree(full_path, branch)
        else:
            tree.add(item)

root_path = r".\Data"
tree = Tree(f"[bold green]{os.path.basename(root_path)}/")
build_tree(root_path, tree)
print(tree)

#### 1.3 Process all csv (columns change to rows)

In [14]:
import pandas as pd
from glob import glob
from pathlib import Path
from tqdm import tqdm

# 设置你的输入文件路径和输出路径
input_dir_root = "./Data"
csvs = glob(os.path.join(input_dir_root, '*/*.csv'))
# output_csv_path = "./20140514_long.csv"

for csv in tqdm(csvs):
    output_csv_path = os.path.join(Path(csv).parent, f"{Path(csv).stem}_long.csv")

    # 读取原始宽格式 CSV 数据
    df = pd.read_csv(csv)

    # 将从第4列（索引为3）开始的所有列名作为城市列
    city_columns = df.columns[3:]

    # 使用 pd.melt 进行宽转长
    long_df = pd.melt(
        df,
        id_vars=["date", "hour", "type"],
        value_vars=city_columns,
        var_name="city",
        value_name="value"
    )

    # === 去除 value 为 NaN 的行 ===
    long_df = long_df.dropna(subset=["value"])

    # 保存为新的长格式 CSV 文件
    long_df.to_csv(output_csv_path, index=False, encoding="utf-8")

print(f"✅ 转换成功，保存为：{output_csv_path}")


100%|██████████| 3958/3958 [07:37<00:00,  8.64it/s]

✅ 转换成功，保存为：Data/2016/20160810_long.csv


#### 1.4 Add auxiliary csv

In [ ]:
import os
import pandas as pd
import json
from glob import glob
from tqdm import tqdm
from pathlib import Path

# 设置你的输入文件路径和输出路径
input_dir_root = "./Data"
csvs = glob(os.path.join(input_dir_root, '*/*long.csv'))

valid_dates = dict()
valid_cities = dict()

for csv in tqdm(csvs):
    date = Path(csv).stem.strip('_long')
    valid_dates[date] = list()
    
    df = pd.read_csv(csv)
    cities = set(df['city'])
    for city in cities:
        valid_cities[city] = date
    
    valid_dates[date].append(list(cities))

with open('./dates_to_cities_index.json', 'w+') as f:
    json.dump(valid_dates, f, ensure_ascii=False, indent=4)
with open('./cities_to_dates_index.json', 'w+') as f:
    json.dump(valid_cities, f, ensure_ascii=False, indent=4)

100%|██████████| 3958/3958 [02:40<00:00, 24.67it/s]


#### 2.1 Due to large amount of data, we average the air quality metrics from each hour in one day to only one value.

In [4]:
import pandas as pd
import os
from glob import glob
from tqdm import tqdm

data_root = r".\Data"
csvs = glob(os.path.join(data_root, '*/*.csv'))
for csv in tqdm(csvs):

    # 读取原始 CSV 文件
    df = pd.read_csv(csv)

    # 只保留我们感兴趣的 type
    target_types = ['AQI', 'PM2.5', 'PM10', 'SO2', 'NO2', 'O3', 'CO']
    df = df[df['type'].isin(target_types)]

    # 计算每个城市、每种污染物的当天平均值
    avg_df = df.groupby(['city', 'type'])['value'].mean().reset_index()
    avg_df['value'] = avg_df['value'].round(2)

    # 保存为新的 CSV 文件
    avg_df.to_csv(csv, index=False)

print("处理完成")


100%|██████████| 3958/3958 [07:56<00:00,  8.31it/s]

处理完成


#### 2.2 Average again on months.

In [10]:
import pandas as pd
import os
import glob

def process_monthly_avg(data_root='Data', year='2015'):
    year_path = os.path.join(data_root, year)
    months = {f'{m:02d}': [] for m in range(1, 13)}

    # 找出所有 *_summary.csv 文件
    all_files = glob.glob(os.path.join(year_path, '*.csv'))

    # 将文件按月份分类
    for file in all_files:
        basename = os.path.basename(file)
        if len(basename) == 12:  # e.g., 20150101_summary.csv
            month = basename[4:6]
            if month in months:
                months[month].append(file)

    # 遍历每个月的文件进行聚合
    for month, file_list in months.items():
        if not file_list:
            continue  # 当前月份无文件，跳过

        monthly_df = pd.concat([pd.read_csv(f) for f in file_list])
        monthly_avg = monthly_df.groupby(['city', 'type'])['value'].mean().reset_index()
        monthly_avg['value'] = monthly_avg['value'].round(2)  # 保留两位小数

        # 输出文件保存
        output_file = os.path.join(year_path, f'{year}_{month}.csv')
        monthly_avg.to_csv(output_file, index=False)
        print(f'保存 {output_file} 完成，共处理 {len(file_list)} 个文件')

# 运行示例：处理2015年数据
years = ['2014', ]
# '2015', '2016', '2017', '2018', '2019', '2020', '2021', '2022', '2023', '2024', '2025']
for year in years:
    process_monthly_avg(data_root='Data', year=year)


保存 Data\2014\2014_05.csv 完成，共处理 19 个文件
保存 Data\2014\2014_06.csv 完成，共处理 30 个文件
保存 Data\2014\2014_07.csv 完成，共处理 22 个文件
保存 Data\2014\2014_08.csv 完成，共处理 31 个文件
保存 Data\2014\2014_09.csv 完成，共处理 30 个文件
保存 Data\2014\2014_10.csv 完成，共处理 31 个文件
保存 Data\2014\2014_11.csv 完成，共处理 30 个文件
保存 Data\2014\2014_12.csv 完成，共处理 31 个文件


#### 2.3 Merge months data into one file for each year.

In [13]:
import pandas as pd
import os
import glob

def merge_yearly_monthly_averages(data_root='Data', year='2014'):
    year_path = os.path.join(data_root, year)
    pattern = os.path.join(year_path, f'{year}_*.csv')
    files = sorted(glob.glob(pattern))

    all_data = []

    for file in files:
        df = pd.read_csv(file)
        # 提取月份（例如从 2014_05_monthly_avg.csv 提取 "201405"）
        filename = os.path.basename(file)
        month_str = filename.split('_')[1]
        df.insert(1, 'month', f'{year}{month_str}')
        all_data.append(df)

    if all_data:
        combined_df = pd.concat(all_data, ignore_index=True)
        output_file = os.path.join(year_path, f'{year}_all.csv')
        combined_df.to_csv(output_file, index=False)
        print(f'保存合并结果至：{output_file}')
    else:
        print(f'未在 {year_path} 找到任何月平均文件')

# 示例调用
years = ['2015', '2016', '2017', '2018', '2019', '2020', '2021', '2022', '2023', '2024', '2025']
for year in years:
    merge_yearly_monthly_averages(data_root='Data', year=year)


保存合并结果至：Data\2015\2015_all.csv
保存合并结果至：Data\2016\2016_all.csv
保存合并结果至：Data\2017\2017_all.csv
保存合并结果至：Data\2018\2018_all.csv
保存合并结果至：Data\2019\2019_all.csv
保存合并结果至：Data\2020\2020_all.csv
保存合并结果至：Data\2021\2021_all.csv
保存合并结果至：Data\2022\2022_all.csv
保存合并结果至：Data\2023\2023_all.csv
保存合并结果至：Data\2024\2024_all.csv
保存合并结果至：Data\2025\2025_all.csv


#### 2.3 Merge all annual files into one.

In [15]:
import pandas as pd
import os
import glob

def merge_all_years(data_root='Data', output_file='all_data.csv'):
    pattern = os.path.join(data_root, '*/*_all.csv')
    files = sorted(glob.glob(pattern))

    all_data = []

    for file in files:
        df = pd.read_csv(file)
        # 重命名 'month' 列为 'date'
        df.rename(columns={'month': 'date'}, inplace=True)
        all_data.append(df)

    if all_data:
        combined_df = pd.concat(all_data, ignore_index=True)
        combined_df.to_csv(output_file, index=False)
        print(f'整合完成，保存为：{output_file}')
    else:
        print(f'未在 {data_root} 中找到 *_all_months.csv 文件')

# 示例调用
merge_all_years(data_root='Data', output_file='./Data/all_data.csv')


整合完成，保存为：./Data/all_data.csv


#### 3. Translate all Chinese into English

In [ ]:
import pandas as pd
import asyncio
import nest_asyncio
from googletrans import Translator
from tqdm import tqdm
from glob import glob
from pathlib import Path
nest_asyncio.apply()

translator = Translator()
csv = r"./Data/all_data.csv"
df = pd.read_csv(csv)
column = 'city'

# 异步翻译函数
async def translate_text(text, src='zh-cn', dest='en'):
    try:
        result = await translator.translate(text, src=src, dest=dest)
        return result.text
    except Exception as e:
        print(f"翻译失败: {text}，错误：{e}")
        return None

# 主异步函数，处理整个 DataFrame 的翻译
async def translate_column(df, column_name, new_column_name):
    tasks = [translate_text(row[column_name]) for _, row in df.iterrows()]
    results = await asyncio.gather(*tasks)
    df[new_column_name] = results
    return df

loop = asyncio.get_event_loop()
df_translated = asyncio.run(translate_column(df, column, 'city_en'))
df_translated.to_csv(r"./Data/all_data_trans.csv", index=False, encoding='utf-8')


#### 3.1 For Bubble chart, we need to create a new json file

In [1]:
import pandas as pd

# === 1. 读取合并后的 CSV 文件 ===
file_path = "./Data/merged_all_data_en.csv"  # 替换为你的实际路径
df = pd.read_csv(file_path)

# === 2. 转换日期格式，并提取 "年月" ===
dates = df['date'].copy().apply(lambda x: str(x)) 
df['date'] = pd.to_datetime(dates, format="%Y%m", errors='coerce')
df['year_month'] = df['date'].dt.to_period('M').astype(str)

# === 3. 获取所有污染物类型 ===
pollutants = df['type'].unique()

# === 4. 针对每种污染物，计算每个省 + 月份的平均值 ===
agg_frames = []
for pollutant in pollutants:
    sub_df = df[df['type'] == pollutant]
    grouped = sub_df.groupby(['province', 'year_month'])['value'].mean().reset_index()
    grouped.rename(columns={'value': pollutant}, inplace=True)
    agg_frames.append(grouped)

# === 5. 依次合并多个污染物的聚合结果 ===
from functools import reduce
merged_pollution = reduce(lambda left, right: pd.merge(left, right, on=['province', 'year_month'], how='outer'), agg_frames)

# === 6. 添加静态字段：人口、GDP、地区等 ===
static_fields = df[['province', 'population', 'gdp', 'region', 'climate_type', 'is_coastal']].drop_duplicates('province')
final = pd.merge(merged_pollution, static_fields, on='province', how='left')

# === 7. 可选导出为 JSON 或 CSV 文件 ===
final.to_csv("./Data/bubble_data_full_en.csv", index=False)
# final.to_json("bubble_data_full.json", orient="records", force_ascii=False)

# === 8. 输出预览前几行确认结果 ===
# print(final.head())


#### Final Translation

In [14]:
from translate import Translator
import pandas as pd
import os
from tqdm import tqdm
from glob import glob
from pathlib import Path

csv_path = r"./Data/merged_all_data_en.csv"
out = r"./Data/merged_all_data_en2.csv"
df = pd.read_csv(csv_path)
column = 'province'
translator = Translator(from_lang='zh', to_lang='en')
cities = list(set(df[column]))
translated_cities = dict()
for city in tqdm(cities):
    transtext = translator.translate(city)
    transtext = transtext.split(' ')[0]
    print(f"翻译 {city} 为 {transtext}")
    translated_cities[city] = transtext
df[column] = df[column].map(translated_cities)
df.to_csv(out, index=False, encoding='utf-8')

  3%|▎         | 1/32 [00:00<00:23,  1.34it/s]

翻译 河南省 为 Henan


  6%|▋         | 2/32 [00:01<00:22,  1.35it/s]

翻译 山西省 为 Shanxi


  9%|▉         | 3/32 [00:02<00:21,  1.32it/s]

...

 19%|█▉        | 6/32 [00:06<00:34,  1.31s/it]

翻译 四川省 为 Sichuan


 22%|██▏       | 7/32 [00:08<00:28,  1.16s/it]

翻译 云南省 为 Yunnan


AttributeError: 'float' object has no attribute 'expandtabs'